## Classifier model 
(since linear regression doesnt seem to work when we only look at the weather)

#### Load dataframe, sort to 1984-2023, change date column into date format

In [ ]:
#import both dataframe (fire and weather), sort to 1984-2023
import pandas as pd
weather = pd.read_csv(r'weather_pivoted.csv')
fire = pd.read_csv(r'fire data with 83 firestn.csv')


weather['date']= pd.to_datetime(weather['date'])
fire['ALARM_DATE'] = pd.to_datetime(fire['ALARM_DATE'])

weather = weather.loc[(weather['date'] >= '1984-01-01') & (weather['date'] <= '2023-12-31')]


#weather.set_index(['date'])
#fire.set_index(['ALARM_DATE'])



C:\Users\natal\AppData\Local\Temp\ipykernel_35960\737385975.py:7: UserWarning: Parsing dates in %d/%m/%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  fire['ALARM_DATE'] = pd.to_datetime(fire['ALARM_DATE'])


### Merge 2 dataframe and add a column to indicate fire occurence (1/0)

In [ ]:
#merge dataframes

weather_fire_merged = weather.merge(fire, left_on='date', right_on='ALARM_DATE', how='left')

#adding a column to indicate whether there is a fire on the date
weather_fire_merged['Fire_occur'] = weather_fire_merged['GIS_ACRES'].notna().astype(int)




In [28]:
weather_fire_merged= pd.read_csv(r'weather_fire_merged pivoted.csv')
a = weather_fire_merged.isnull().sum()
weather_fire_merged = weather_fire_merged.drop(columns=weather_fire_merged.columns[weather_fire_merged.isnull().sum() > 400000])

# Drop specified columns
weather_fire_class = weather_fire_merged.drop(columns=['GIS_ACRES', 'Shape__Area', 'Shape__Length'])



pd.merge on column is too slow, try making id and date index now

### Classification model: Random forest classifier

** have not done it so this is just a template: 260225**

In [73]:
print(weather_fire_merged['Fire_occur'].value_counts())



Fire_occur
0    270065
1    225800
Name: count, dtype: int64


In [71]:
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.model_selection import train_test_split

# Ensure ALARM_DATE is in datetime format
weather_fire_merged['ALARM_DATE'] = pd.to_datetime(weather_fire_merged['ALARM_DATE'])

# Feature Engineering for Time Series Classification
weather_fire_merged['month'] = weather_fire_merged['ALARM_DATE'].dt.month  # Capture seasonality
weather_fire_merged['year'] = weather_fire_merged['ALARM_DATE'].dt.year    # Capture trend

# Define Features and Target
X = weather_fire_merged.loc[:, 'PRCP':'TOBS']  # Select all columns from PRCP to TOBS
X['month'] = weather_fire_merged['month']      # Add month as a feature
y = weather_fire_merged['Fire_occur']          # Target variable (binary)

# Ensure `y` is numeric (in case it's not already)
y = y.astype(int)  # 1 = Fire occurred, 0 = No Fire

# Split the data into train and test (stratified by 'Fire_occur' to keep class balance)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# Check the class distribution in the training and testing sets
print("Train class distribution:\n", y_train.value_counts())
print("Test class distribution:\n", y_test.value_counts())

# Define and Train the RandomForestClassifier model
model = RandomForestClassifier(random_state=42, n_estimators=100, class_weight='balanced')
model.fit(X_train, y_train)

# Make Predictions
y_pred = model.predict(X_test)

# Evaluate the Classification Model
print("🔥 Accuracy:", accuracy_score(y_test, y_pred))
print("\n📊 Classification Report:\n", classification_report(y_test, y_pred))
print("\n🧩 Confusion Matrix:\n", confusion_matrix(y_test, y_pred))


Train class distribution:
 Fire_occur
0    216052
1    180640
Name: count, dtype: int64
Test class distribution:
 Fire_occur
0    54013
1    45160
Name: count, dtype: int64
🔥 Accuracy: 1.0

📊 Classification Report:
               precision    recall  f1-score   support

           0       1.00      1.00      1.00     54013
           1       1.00      1.00      1.00     45160

    accuracy                           1.00     99173
   macro avg       1.00      1.00      1.00     99173
weighted avg       1.00      1.00      1.00     99173


🧩 Confusion Matrix:
 [[54013     0]
 [    0 45160]]


In [72]:
from sklearn.model_selection import cross_val_score

# Perform cross-validation
cv_scores = cross_val_score(model, X, y, cv=5, scoring='accuracy')
print("Cross-validated accuracy scores:", cv_scores)
print("Average cross-validated accuracy:", cv_scores.mean())


Cross-validated accuracy scores: [1. 1. 1. 1. 1.]
Average cross-validated accuracy: 1.0


### now that looks too good to be true: we look at the feature importance to see which one is the dominating factor

### Month(seasonality) contribute the most: try dropping month and see if the model can still perform well

Seasonal encoding: You can use sinusoidal encoding (sine and cosine transformations) for month and day features to capture periodicity without allowing the raw values to dominate. This allows the model to interpret months as a continuous cycle.
<-> i tried and the model rely too much on seasonal pattern(feature importance) -> now the below code predict based on weather features only

In [ ]:
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.model_selection import train_test_split

# Ensure ALARM_DATE is in datetime format
weather_fire_merged['ALARM_DATE'] = pd.to_datetime(weather_fire_merged['ALARM_DATE'])

# Feature Engineering for Time Series Classification
#weather_fire_merged['month'] = weather_fire_merged['ALARM_DATE'].dt.month  # Capture seasonality
#weather_fire_merged['year'] = weather_fire_merged['ALARM_DATE'].dt.year    # Capture trend
weather_fire_merged['month_sin'] = np.sin(2 * np.pi * weather_fire_merged['month'] / 12)
weather_fire_merged['month_cos'] = np.cos(2 * np.pi * weather_fire_merged['month'] / 12)

# Define Features and Target
X = weather_fire_merged.loc[:, 'PRCP':'TOBS']  # Select all columns from PRCP to TOBS
#X = X.assign(month_sin=weather_fire_merged['month_sin'], month_cos=weather_fire_merged['month_cos'])
y = weather_fire_merged['Fire_occur']          # Target variable (binary)

# Ensure `y` is numeric (in case it's not already)
y = y.astype(int)  # 1 = Fire occurred, 0 = No Fire

# Split the data into train and test (stratified by 'Fire_occur' to keep class balance)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# Check the class distribution in the training and testing sets
print("Train class distribution:\n", y_train.value_counts())
print("Test class distribution:\n", y_test.value_counts())

# Define and Train the RandomForestClassifier model
model = RandomForestClassifier(random_state=42, n_estimators=100, class_weight='balanced')




# Train the model with the modified data (without the 'month' column)
model.fit(X_train, y_train)

# Make predictions on the test set
y_pred = model.predict(X_test)


# Evaluate the Classification Model
print("🔥 Accuracy:", accuracy_score(y_test, y_pred))
print("\n📊 Classification Report:\n", classification_report(y_test, y_pred))
print("\n🧩 Confusion Matrix:\n", confusion_matrix(y_test, y_pred))


Train class distribution:
 Fire_occur
0    216052
1    180640
Name: count, dtype: int64
Test class distribution:
 Fire_occur
0    54013
1    45160
Name: count, dtype: int64
Accuracy without month feature: 0.7683744567573836
🔥 Accuracy: 0.7683744567573836

📊 Classification Report:
               precision    recall  f1-score   support

           0       0.80      0.76      0.78     54013
           1       0.73      0.78      0.75     45160

    accuracy                           0.77     99173
   macro avg       0.77      0.77      0.77     99173
weighted avg       0.77      0.77      0.77     99173


🧩 Confusion Matrix:
 [[41131 12882]
 [10089 35071]]


In [41]:
# Test set evaluation
y_pred = model.predict(X_test)
print("Test Accuracy:", accuracy_score(y_test, y_pred))


Test Accuracy: 0.7683744567573836


### Feature importance

In [69]:
feature_importances = model.feature_importances_
features = X.columns
sorted_idx = np.argsort(feature_importances)

print("Feature importances (sorted):")
for idx in sorted_idx:
    print(f"{features[idx]}: {feature_importances[idx]}")


Feature importances (sorted):
ACMH: 0.0
WT01: 0.0
WT02: 0.0
WT03: 0.0
WT04: 0.0
WT05: 0.0
WT06: 0.0
WT07: 0.0
WT08: 0.0
WT09: 0.0
WT10: 0.0
WT11: 0.0
WT13: 0.0
WT14: 0.0
WT16: 0.0
WT18: 0.0
TSUN: 0.0
WT19: 0.0
WT22: 0.0
ACSH: 0.0
MDEV: 1.796917741213375e-08
DAEV: 1.9384073495767898e-08
DAWM: 6.550768589264064e-07
MDWM: 1.8340557550621575e-06
DASF: 2.8658421332982403e-06
MDSF: 2.230198366127509e-05
WESF: 4.792322844308201e-05
DAPR: 0.000361493342823275
MDPR: 0.0010239591375674154
ADPT: 0.001123792018748994
FMTM: 0.0012638044784920054
ASTP: 0.0014472217750699074
RHMN: 0.0014599672910270384
ASLP: 0.0015075731379648512
RHMX: 0.0016223254783401398
AWBT: 0.002225762355102353
RHAV: 0.0024182546125888615
WDFG: 0.003241405112120282
WSF2: 0.003661813451163019
WSFG: 0.0037357971830394277
WSF5: 0.003965164011390282
WDF2: 0.004009159301187329
WDF5: 0.004230749655858599
AWND: 0.007157029462290783
MNPN: 0.007220720737543712
PGTM: 0.007430125864413406
WESD: 0.007561139340960549
SNOW: 0.007847634525315

## with all the features, with feature importance (finalised)

In [43]:
weather_fire_merged_all= pd.read_csv(r'weather_fire_merged pivoted.csv')


In [74]:
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.model_selection import train_test_split

# Ensure ALARM_DATE is in datetime format
weather_fire_merged_all['ALARM_DATE'] = pd.to_datetime(weather_fire_merged_all['ALARM_DATE'])

# Feature Engineering for Time Series Classification
weather_fire_merged_all['month'] = weather_fire_merged_all['ALARM_DATE'].dt.month  # Capture seasonality
weather_fire_merged_all['year'] = weather_fire_merged_all['ALARM_DATE'].dt.year    # Capture trend

# Define Features and Target
X = weather_fire_merged_all.loc[:, 'ACMH':'WT22']  # Select all columns from ACMH to WT22
y = weather_fire_merged_all['Fire_occur']          # Target variable (binary)

# Ensure `y` is numeric (in case it's not already)
y = y.astype(int)  # 1 = Fire occurred, 0 = No Fire

# Store ALARM_DATE separately for reference
dates = weather_fire_merged_all['ALARM_DATE']

# Split the data into train and test (stratified by 'Fire_occur' to keep class balance)
X_train, X_test, y_train, y_test, train_dates, test_dates = train_test_split(
    X, y, dates, test_size=0.2, random_state=42, stratify=y
)

# Check the class distribution in the training and testing sets
print("Train class distribution:\n", y_train.value_counts())
print("Test class distribution:\n", y_test.value_counts())

# Define and Train the RandomForestClassifier model
model = RandomForestClassifier(random_state=42, n_estimators=50, class_weight='balanced')
model.fit(X_train, y_train)

# Make Predictions
y_pred = model.predict(X_test)

# Evaluate the Classification Model
print("🔥 Accuracy:", accuracy_score(y_test, y_pred))
print("\n📊 Classification Report:\n", classification_report(y_test, y_pred))
print("\n🧩 Confusion Matrix:\n", confusion_matrix(y_test, y_pred))



Train class distribution:
 Fire_occur
0    216052
1    180640
Name: count, dtype: int64
Test class distribution:
 Fire_occur
0    54013
1    45160
Name: count, dtype: int64
🔥 Accuracy: 0.7777116755568552

📊 Classification Report:
               precision    recall  f1-score   support

           0       0.81      0.78      0.79     54013
           1       0.74      0.78      0.76     45160

    accuracy                           0.78     99173
   macro avg       0.78      0.78      0.78     99173
weighted avg       0.78      0.78      0.78     99173


🧩 Confusion Matrix:
 [[41976 12037]
 [10008 35152]]


### OKAY on top of the prev code: lets try XGboost to see if it perform better

In [48]:
import xgboost as xgb
model = xgb.XGBClassifier(random_state=42)  # Handle class imbalance
model.fit(X_train, y_train)

y_pred = model.predict(X_test)
print("🔥 XGBoost Accuracy:", accuracy_score(y_test, y_pred))



🔥 XGBoost Accuracy: 0.760327911830841


## improve accuracy by adding features 

okay it doesnt work (0.76, RFC has 0.77)
scrap this

In [56]:
# Create an interaction term between temperature (TAVG) and precipitation (PRCP)
weather_fire_merged_all['temp_precip_interaction'] = weather_fire_merged_all['TAVG'] * weather_fire_merged_all['PRCP']



### add distance between fire and weather stations
scrap this

In [53]:
coord_data = pd.read_csv(r'closest_df with fire weather coord .csv')
weather_fire_merged_all['distance_to_forest'] = coord_data['distance']
weather_fire_merged_all['month_sin'] = np.sin(2 * np.pi * weather_fire_merged['month'] / 12)
weather_fire_merged_all['month_cos'] = np.cos(2 * np.pi * weather_fire_merged['month'] / 12)
